In [9]:
# %load std_libraries.py
#Indicate operating environment and import core modules
import os 
location_input = input("what computer are you on? a = Ben's laptop, b = gpucluster, c = Ben's desktop, d = other")
location_dict = {'a': "C:\\Users\\BMH_work\\", 'b': "/home/heineike/",
                 'c': "C:\\Users\\Ben\\Documents\\", 'd':'you need to add your location to the location_dict'}
home_dir = location_dict[location_input]
print("home directory is " + home_dir)
base_dir = home_dir + os.path.normpath('github/y1000plus_tools') + os.sep
print("y1000plus_tools dir is " + base_dir )
y1000plus_dir_options = {'b':home_dir + os.path.normpath("genomes/y1000plus") + os.sep, 
                         'c': home_dir + os.path.normpath('github/expression_broad_data/expression_data/promoter_phylogenies/y1000plus') + os.sep
                        }
y1000plus_dir = y1000plus_dir_options[location_input]
print("y1000plus data dir is " + y1000plus_dir)

import sys

if sys.path[-1] != base_dir:
    sys.path.append(base_dir)
    print("Added " + base_dir + " to path: " )
    print(sys.path)

print("Importing y1000plus_tools.py")
from core import y1000plus_tools
%load_ext autoreload
%autoreload 2


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from Bio.Seq import Seq
from Bio.Alphabet import generic_dna
from Bio import SeqIO
from ete3 import Tree
#ete3 is not officially supported on windows, and so must be loaded via pip: 
# pip install -U https://github.com/etetoolkit/ete/archive/qt5.zip
# ref: https://groups.google.com/forum/#!topic/etetoolkit/6NblSBPij4o
#20181031: got this error message: twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.


what computer are you on? a = Ben's laptop, b = gpucluster, c = Ben's desktop, d = otherb
home directory is /home/heineike/
y1000plus_tools dir is /home/heineike/github/y1000plus_tools/
y1000plus genomes dir is /home/heineike/genomes/y1000plus/
Importing y1000plus_tools.py
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
y1000_species = pd.read_csv(y1000plus_dir + "y1000_species_table.csv", index_col=0)

#Select only clades of interest: 
clades_of_interest = {'Saccharomycetaceae','Saccharomycodaceae','Phaffomycetaceae','CUG-Ser2','CUG-Ser1','Pichiaceae','CUG-Ala'}
y1000_species_subset = y1000_species[y1000_species['Major clade'].isin(clades_of_interest)]

In [24]:
#genes of interest: 

#NOtice how many of these genes are very close on the genome MPC3, SOL4, ENO1, GND2


gois = ['YGR043C','YLR354C', #Start strong Msn2/4 regulation NQM1/TAL1
       'YGR256W', 'YHR183W', #GND2/GND1 - Missing genes swapped between expression and stres. 
        'YMR105C', 'YKL127W', #PGM2/PGM1 - still has STRE but expression goes away
        #'YMR285C','YML118W', #NGL2/NGL3 - Poly-A RNA exonuclease - different OGs 
        'YGR254W','YHR174W', #ENO1/ENO2
        'YGR248W','YHR163W', #SOL4/SOL3 - strong PDS signal
        'YGR243W','YHR162W', #MPC3/MPC2 - Start Weak/no Msn2/4 effect: 
        'YJL164C','YKL166C', #TPK1/TPK3   (but GIS1/RPH1 had an effect and there is a strong PDS) 
        'YDL021W','YOL056W' #GPM2/GPM3 - #Start average Msn2/4 effect: expression data missing ortholog for high, I'm missing ortholog from low 
        #'YPL230W','YMR182C' #USV1/RGM1 - Different OGs
      ]





In [25]:
y1000_ids = dict(scer_lookup.loc[gois,'y1000_id'])
y1000_id_list = [y1000_ids[goi] for goi in gois]
y1000_id_list

['110_821',
 '110_1753',
 '110_4266',
 '110_326',
 '110_1562',
 '110_689',
 '110_757',
 '110_4425',
 '110_4019',
 '110_1058',
 '110_2707',
 '110_3574',
 '110_2527',
 '110_1854',
 '110_1259',
 '110_2716']

In [26]:
#Make promoter files for various genes of interest
#Load S.Cer lookup table: 
scer_lookup_fname = y1000plus_dir + os.path.normpath('id_lookups/saccharomyces_cerevisiae.csv')

scer_lookup = pd.read_csv(scer_lookup_fname, index_col=0)
# target_y1000_id = scer_lookup.loc[goi,'y1000_id']

y1000_ids = dict(scer_lookup.loc[gois,'y1000_id'])
y1000_id_list = [y1000_ids[goi] for goi in gois]
y1000_id_list

goi_og_lookup, og_genes_lookup = y1000plus_tools.make_og_genes_lookup(y1000_id_list, y1000_species_subset)

more than one goi in same orthogroup :OG1193 found_genes: 
['110_757', '110_4425']
more than one goi in same orthogroup :OG1302 found_genes: 
['110_689', '110_1562']
more than one goi in same orthogroup :OG1488 found_genes: 
['110_326', '110_4266']
more than one goi in same orthogroup :OG1667 found_genes: 
['110_1753', '110_821']
more than one goi in same orthogroup :OG1859 found_genes: 
['110_2707', '110_3574']
more than one goi in same orthogroup :OG3991 found_genes: 
['110_1058', '110_4019']
more than one goi in same orthogroup :OG4796 found_genes: 
['110_1854', '110_2527']
more than one goi in same orthogroup :OG6005 found_genes: 
['110_2716', '110_1259']


In [27]:
[goi_og_lookup[y1000_ids[goi]] for goi in gois]

['OG1667',
 'OG1667',
 'OG1488',
 'OG1488',
 'OG1302',
 'OG1302',
 'OG1193',
 'OG1193',
 'OG3991',
 'OG3991',
 'OG1859',
 'OG1859',
 'OG4796',
 'OG4796',
 'OG6005',
 'OG6005']

In [97]:
#Consolidate pairs with orthogroups
goi_pair_og_lookup =  {'NQM1_TAL1': 'OG1667', 
                     'GND2_GND1': 'OG1488', 
                     'PGM2_PGM1': 'OG1302', 
                     'ENO1_ENO2': 'OG1193', 
                     'SOL4_SOL3': 'OG3991', 
                     'MPC3_MPC2': 'OG1859', 
                     'TPK1_TPK3': 'OG4796',
                     'GPM2_GPM3': 'OG6005'
                    }


L_prom = 1000

for goi_pair, og in goi_pair_og_lookup.items(): 
    print(goi_pair)
    og_genes =  og_genes_lookup[og]
    
    #For a given orthogroup extract promoters

    y1000plus_tools.extract_promoters(L_prom, og, og_genes, y1000_species_subset, fname_string=goi_pair)
    print(goi_pair + ' complete')

what computer are you on? a = Ben's laptop, b = gpucluster, c = Ben's desktop, d = otherb
home directory is /home/heineike/
y1000plus_tools dir is /home/heineike/github/y1000plus_tools/
y1000plus genomes dir is /home/heineike/genomes/y1000plus/
TPK1_TPK3
promoter region extends past the scaffold, genome_name = yHMPu5000035703_cyberlindnera_americana_160613 Gene = Seq_4055, L_prom = 1000
promoter region extends past the scaffold, genome_name = yHMPu5000034623_pichia_norvegensis_160519 Gene = Seq_641, L_prom = 1000
scaffold ends at the end of the gene, genome_name = yHMPu5000034623_pichia_norvegensis_160519 Gene = Seq_641
promoter region extends past the scaffold, genome_name = yHMPu5000034627_pichia_heedii_160519 Gene = Seq_3722, L_prom = 1000
promoter region extends past the scaffold, genome_name = yHMPu5000035318_hyphopichia_heimii_160928 Gene = Seq_422, L_prom = 1000
TPK1_TPK3 complete
NQM1_TAL1
promoter region extends past the scaffold, genome_name = yHMPu5000034963_hanseniaspora_cl

In [ ]:
#Find STRE/PDS/TATA box in promoters

In [ ]:
#Make phylogenetic trees for various genes of interest

In [ ]:
#Visualize STRE and phylogenetic tree

#Arrange promoters according to gene's phylogenetic tree
